In [39]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# DATA

In [40]:
app_train = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_train.csv')
app_test = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\app_test.csv')
prev_app = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\prev_app.csv')
pos_cash = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\pos_cash.csv')
bureau = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\bureau_balance_merge_bureau.csv')
ccb = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\credit_card_balance_final.csv')
inp = pd.read_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\installments_payments_final.csv').drop(columns='Unnamed: 0')

In [41]:
inp = inp.rename(columns={'SK_ID_CURR_': 'SK_ID_CURR'})

In [42]:
full_app = pd.concat([app_train, app_test], axis = 0)
# full_app[(full_app['TARGET'] == 0) | (full_app['TARGET'] == 1)]

# Join All

In [43]:
full_train = pd.merge(full_app, bureau, on= 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,prev_app, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,inp, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,ccb, on = 'SK_ID_CURR', how = 'left')
full_train = pd.merge(full_train,pos_cash, on = 'SK_ID_CURR', how = 'left')

# Fill nan

In [44]:
## check null
def check_missed_values(df):
    df_null_percentage = df.isnull().sum() / df.shape[0] * 100
    df_null_percentage = df_null_percentage.drop(df_null_percentage[df_null_percentage == 0].index).sort_values(ascending= False).reset_index()
    df_null_percentage.columns = ["Feature", "Percentage"]
    return df_null_percentage

null_df = check_missed_values(full_train)
null_df

,Feature,Percentage
0,AMT_PAYMENT_CURRENT_MEAN,80.143800
1,AMT_PAYMENT_CURRENT_MIN,80.143800
2,AMT_PAYMENT_CURRENT_MAX,80.143800
3,AMT_DRAWINGS_ATM_CURRENT_MAX,80.117784
4,AMT_DRAWINGS_OTHER_CURRENT_MAX,80.117784
...,...,...
172,COUNT_TIMES_OVER_DUE_MIN,5.883042
173,NAME_TYPE_SUITE,0.420148
174,EXT_SOURCE_2,0.214626
175,AMT_ANNUITY,0.003902


In [45]:
drop_null_col =null_df[null_df["Percentage"] > 70]["Feature"].tolist()
full_train.drop(drop_null_col, axis=1, inplace= True)

# drop cot toan la gia tri 0

In [46]:
numeric_df = full_train._get_numeric_data()
numeric_df.drop([ "SK_ID_CURR", "TARGET"], axis= 1, inplace= True)

count_list =[]
for i in numeric_df.columns:
    fre_count = pd.Series(numeric_df[i].value_counts(normalize= True))
    count_list.append([i, fre_count.index[0], fre_count.values[0] * 100])

frequent_df = pd.DataFrame(count_list)
frequent_df.columns = ["Feature", "Value", "Frequent"]
frequent_df = frequent_df.sort_values(by= "Frequent",ascending= False)
to_drop = frequent_df[frequent_df["Frequent"] > 75]["Feature"].tolist()

full_train.drop(to_drop, axis=1, inplace= True)
full_train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_ANNUITY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,...,CNT_INSTALMENT_MAX,CNT_INSTALMENT_MIN,CNT_INSTALMENT_FUTURE_SUM,CNT_INSTALMENT_FUTURE_MEAN,CNT_INSTALMENT_FUTURE_MAX,PAYMENT_COUNTS_SUM,PAYMENT_COUNTS_MEAN,PAYMENT_COUNTS_MAX,PAYMENT_COUNTS_MIN,COMPLETED_STATUS_SUM
0,278621,0.0,Cash loans,F,0,270000.0,35698.5,Family,State servant,Higher education,...,12.0,6.0,1.0,0.333333,1.0,28.0,9.333333,12.0,8.0,2.0
1,139008,0.0,Cash loans,F,0,135000.0,29686.5,Unaccompanied,Working,Secondary / secondary special,...,12.0,1.0,3.0,1.000000,3.0,21.0,7.000000,10.0,3.0,2.0
2,138348,0.0,Cash loans,M,0,121500.0,21865.5,Unaccompanied,Working,Secondary / secondary special,...,24.0,10.0,14.0,2.800000,13.0,66.0,13.200000,18.0,10.0,3.0
3,64140,0.0,Cash loans,M,0,99000.0,27517.5,"Spouse, partner",State servant,Secondary / secondary special,...,10.0,6.0,0.0,0.000000,0.0,83.0,20.750000,55.0,8.0,4.0
4,219374,0.0,Cash loans,M,0,360000.0,42075.0,Unaccompanied,State servant,Higher education,...,10.0,10.0,0.0,0.000000,0.0,11.0,11.000000,11.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,150442,NaN,Cash loans,M,0,171000.0,23089.5,Family,Pensioner,Higher education,...,24.0,2.0,3.0,0.272727,2.0,157.0,14.272727,24.0,3.0,9.0
307507,5217,NaN,Revolving loans,M,2,450000.0,45000.0,Family,Working,Secondary / secondary special,...,24.0,4.0,1.0,0.250000,1.0,54.0,13.500000,25.0,5.0,3.0
307508,260741,NaN,Cash loans,F,0,225000.0,24030.0,Family,Commercial associate,Secondary / secondary special,...,8.0,1.0,1.0,0.125000,1.0,44.0,5.500000,9.0,2.0,7.0
307509,284794,NaN,Cash loans,M,0,121500.0,30357.0,Family,Pensioner,Secondary / secondary special,...,15.0,4.0,0.0,0.000000,0.0,21.0,10.500000,16.0,5.0,2.0


In [47]:
numerical_data = full_train._get_numeric_data().drop(columns = ["SK_ID_CURR", "TARGET"])
cat_cols = [col for col in full_train.columns if col not in numerical_data.columns and col not in ["SK_ID_CURR", "TARGET"]]

In [48]:
# Fill missing values for numerical columns with the mean
numerical_data.fillna(numerical_data.mean(), inplace=True)

# Fill missing values for categorical columns with the mode
for col in cat_cols:
    mode_value = full_train[col].mode()[0]  
    full_train[col].fillna(mode_value, inplace=True)


In [49]:
cat_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',
 'INS_ON_TIME_GRADE_',
 'PAYMENT_GRADE_']

# Transformation

In [50]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
one_hot_encoder = OneHotEncoder
ordinal_encoder = OrdinalEncoder

X_ord = full_train[['INS_ON_TIME_GRADE_','PAYMENT_GRADE_']]
X_one_hot = full_train[['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'ORGANIZATION_TYPE',]]
y = full_train["TARGET"]

X_one_hot = OneHotEncoder().fit_transform(X_one_hot)

In [51]:
X_one_hot = X_one_hot.toarray()
X_cat = pd.DataFrame(X_one_hot)
X_cat

,0,1,2,3,4,5,6,7,8,9,...,103,104,105,106,107,108,109,110,111,112
0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
307507,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
307508,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307509,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [52]:
def check_values(column):
    # Check for infinity
    has_infinity = np.any(np.isinf(column))
    
    # Check for values too large for float64
    too_large_values = np.any(np.abs(column) > np.finfo(np.float64).max)
    
    return has_infinity, too_large_values

# Iterate over columns and check for infinity and large values
def find_col(data):
    cols = []
    for col in data.columns:
        has_infinity, too_large_values = check_values(numerical_data[col])
        
        if has_infinity:
            print(f"Column '{col}' contains infinity.")
            cols.append(col)

        
        if too_large_values:
            print(f"Column '{col}' contains values too large for dtype('float64').")
            if col not in cols:
                cols.append(col)
    return cols

col = find_col(numerical_data)

Column 'PAYS_TO_INS_' contains infinity.
Column 'PAYS_TO_INS_' contains values too large for dtype('float64').


In [53]:
from sklearn.preprocessing import MinMaxScaler

X_num = numerical_data.drop(columns=col, axis=1 )

scaler = MinMaxScaler()
X_num = scaler.fit_transform(X_num)
X_num = pd.DataFrame(X_num)
X_num

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,0.000000,0.002089,0.132924,0.045016,0.510204,0.956149,0.052632,0.312933,0.727773,0.569889,...,0.132530,0.070423,0.007576,0.005556,0.0125,0.091837,0.087719,0.115789,0.073684,0.083333
1,0.000000,0.000935,0.109477,0.107023,0.653061,0.960915,0.052632,0.514238,0.760751,0.569889,...,0.132530,0.000000,0.022727,0.016667,0.0375,0.068027,0.063158,0.094737,0.021053,0.083333
2,0.000000,0.000819,0.078975,0.392880,0.693878,0.960915,0.000000,0.514238,0.377472,0.569889,...,0.277108,0.126761,0.106061,0.046667,0.1625,0.221088,0.128421,0.178947,0.094737,0.125000
3,0.000000,0.000627,0.101018,0.491595,0.530612,0.957102,0.052632,0.514238,0.414298,0.693145,...,0.108434,0.070423,0.000000,0.000000,0.0000,0.278912,0.207895,0.568421,0.073684,0.166667
4,0.000000,0.002858,0.157792,0.039215,0.632653,0.954242,0.052632,0.514238,0.835415,0.603169,...,0.108434,0.126761,0.000000,0.000000,0.0000,0.034014,0.105263,0.105263,0.105263,0.041667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.000000,0.001243,0.083749,0.254009,0.918367,0.000000,0.052632,0.587811,0.655192,0.457380,...,0.277108,0.014085,0.022727,0.004545,0.0250,0.530612,0.139713,0.242105,0.021053,0.375000
307507,0.105263,0.003628,0.169200,0.045016,0.632653,0.958055,0.157895,0.597988,0.469699,0.343066,...,0.277108,0.042254,0.007576,0.004167,0.0125,0.180272,0.131579,0.252632,0.042105,0.125000
307508,0.000000,0.001704,0.087417,0.392880,0.265306,0.953289,0.052632,0.396927,0.579817,0.154091,...,0.084337,0.000000,0.007576,0.002083,0.0125,0.146259,0.047368,0.084211,0.010526,0.291667
307509,0.000000,0.000819,0.112092,0.048852,0.897959,0.000000,0.052632,0.514238,0.692336,0.828218,...,0.168675,0.042254,0.000000,0.000000,0.0000,0.068027,0.100000,0.157895,0.042105,0.083333


In [54]:
full_data = pd.concat([X_num,X_cat],axis = 1)
full_data['SK_ID_CURR'] = full_train['SK_ID_CURR']
full_data['TARGET'] = full_train['TARGET']
full_data

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,SK_ID_CURR,TARGET
0,0.000000,0.002089,0.132924,0.045016,0.510204,0.956149,0.052632,0.312933,0.727773,0.569889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,278621,0.0
1,0.000000,0.000935,0.109477,0.107023,0.653061,0.960915,0.052632,0.514238,0.760751,0.569889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,139008,0.0
2,0.000000,0.000819,0.078975,0.392880,0.693878,0.960915,0.000000,0.514238,0.377472,0.569889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,138348,0.0
3,0.000000,0.000627,0.101018,0.491595,0.530612,0.957102,0.052632,0.514238,0.414298,0.693145,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64140,0.0
4,0.000000,0.002858,0.157792,0.039215,0.632653,0.954242,0.052632,0.514238,0.835415,0.603169,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,219374,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.000000,0.001243,0.083749,0.254009,0.918367,0.000000,0.052632,0.587811,0.655192,0.457380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,150442,NaN
307507,0.105263,0.003628,0.169200,0.045016,0.632653,0.958055,0.157895,0.597988,0.469699,0.343066,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5217,NaN
307508,0.000000,0.001704,0.087417,0.392880,0.265306,0.953289,0.052632,0.396927,0.579817,0.154091,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,260741,NaN
307509,0.000000,0.000819,0.112092,0.048852,0.897959,0.000000,0.052632,0.514238,0.692336,0.828218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,284794,NaN


# SPLIT

In [55]:
train = full_data[(full_data['TARGET'] == 0) | (full_data['TARGET'] == 1)]
test = full_data[~((full_data['TARGET'] == 0) | (full_data['TARGET'] == 1))]


In [56]:
train = train.set_index('SK_ID_CURR')
train.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\train.csv')

In [57]:
test = test.drop(columns='TARGET',axis = 1)
test = test.set_index('SK_ID_CURR')
test.to_csv('D:\\DSEB 63 - NEU\\Năm BA\\Visualization\\train data\\test.csv')